In [1]:
import time
import copy

In [2]:
state = [[[' ',' ',' ',' ',' ',' ',' ',' '],
          [' ','O',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ','-','-',' ',' ',' '],
          [' ',' ','-','-','-','-',' ',' '],
          [' ',' ',' ','-','-',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ','O',' ']], 'X']

K = 5
I_PLAY = 'X'
OPPONENT_PLAYS = 'O'

def staticEval(state):
    # Values should be higher when the states are better for X,
    # lower when better for O.
    N = len(state[0][:][:])
    M = len(state[0][0])
    board = state[0]
    A = 0
    B = 0
    C = 0
    D = 0
    E = 0

    k = 2

    while k <= K:

        # Row
        for row in range(N):
            for col in range(M - k):
                if board[row][col] == I_PLAY:
                    if (all(board[row][col + i] == I_PLAY for i in range(k)) and 
                        board[row].count(I_PLAY) == k):
                        A += 10**(k-1)
                    elif (all(board[row][col + i] == I_PLAY for i in range(k-2)) and  # Check for (k-2) in a row
                          all(board[row][col + i + 1] == ' ' for i in range(k-2)) and  # Check the next position for a potential move
                          board[row][col + k - 1] != OPPONENT_PLAYS):  # Ensure the kth position isn't occupied by the opponent
                        E += 10**(k)

                elif board[row][col] == OPPONENT_PLAYS:
                    if (all(board[row][col + i] == OPPONENT_PLAYS for i in range(k)) and 
                        board[row].count(OPPONENT_PLAYS) == k):
                        B += 10**(k-1)

        # Col
        for row in range(N - k):
            for col in range(M):
                if board[row][col] == I_PLAY:
                    if (all(board[row + i][col] == I_PLAY for i in range(k)) and
                        sum(board[r][col] == I_PLAY for r in range(N)) == k):
                        A += 10**(k-1)
                    elif (all(board[row + i][col] == I_PLAY for i in range(k-2)) and  # Check for (k-2) in a row
                          all(board[row + i + 1][col] == ' ' for i in range(k-2)) and  # Check the next position for a potential move
                          board[row + k -1][col] != OPPONENT_PLAYS):  # Ensure the kth position isn't occupied by the opponent
                        E += 10**(k)
                    
                elif board[row][col] == OPPONENT_PLAYS:
                    if (all(board[row + i][col] == OPPONENT_PLAYS for i in range(k)) and
                        sum(board[r][col] == OPPONENT_PLAYS for r in range(N)) == k):
                        B += 10**(k-1)

        # Diag
        for row in range(N - k):
            for col in range(M - k):
                if board[row][col] == I_PLAY:
                    if (all(board[row + i][col + i] == I_PLAY for i in range(k)) and 
                        sum(board[row + i][col + i] == I_PLAY for i in range(k)) == k):
                        A += 10**(k-1)
                    elif (all(board[row + i][col + i] == I_PLAY for i in range(k-2)) and  # Check for (k-2) in a row
                          all(board[row + i +1][col + i + 1] == ' ' for i in range(k-2)) and  # Check the next position for a potential move
                          board[row + k -1][col + k - 1] != OPPONENT_PLAYS):  # Ensure the kth position isn't occupied by the opponent
                        E += 10**(k)
                elif board[row][col] == OPPONENT_PLAYS:
                    if (all(board[row + i][col + i] == OPPONENT_PLAYS for i in range(k)) and
                        sum(board[row + i][col + i] == OPPONENT_PLAYS for i in range(k)) == k):
                        B += 10**(k-1)

        # Off Diag
        for row in range(N - k):
            for col in range(k - 1, M - 1):
                if board[row][col] == I_PLAY:
                    if (all(board[row + i][col - i] == I_PLAY for i in range(k)) and
                        sum(board[row + i][col - i] == I_PLAY for i in range(k)) == k):
                        A += 10**(k-1)
                    elif (all(board[row + i][col - i] == I_PLAY for i in range(k-2)) and  # Check for (k-2) in a row
                          all(board[row + i +1][col - i - 1] == ' ' for i in range(k-2)) and  # Check the next position for a potential move
                          board[row + k -1][col - k + 1] != OPPONENT_PLAYS):  # Ensure the kth position isn't occupied by the opponent
                        E += 10**(k)
                elif board[row][col] == OPPONENT_PLAYS:
                    if (all(board[row + i][col - i] == OPPONENT_PLAYS for i in range(k)) and
                        sum(board[row + i][col - i] == OPPONENT_PLAYS for i in range(k)) == k):
                        B += 10**(k-1)

        k += 1
    
    # Count singular
    for row in range(N):
        for col in range(M):
            if board[row][col] == I_PLAY:
                if (row == 0 or board[row - 1][col] == ' ') and (col == 0 or board[row][col - 1] == ' '):
                    C += 1
            elif board[row][col] == OPPONENT_PLAYS:
                if (row == 0 or board[row - 1][col] == ' ') and (col == 0 or board[row][col - 1] == ' '):
                    D += 1

    H = A - B + C - D + E

    return H

staticEval(state)

-2

In [3]:
def minimax(state, whoseMove, depthRemaining, pruning=False, alpha=None, beta=None, zHashing=None):

    N = len(state[0][:][:])
    M = len(state[0][0])
    board = state[0]
    
    default_score = 0 # Value of the passed-in state. Needs to be computed.
    if depthRemaining == 0:
        return staticEval(state)
    
    if whoseMove == I_PLAY:
        default_score = - float('inf')
        for row in range(N):
            for col in range(M):
                if board[row][col] == ' ':
                    board[row][col] = I_PLAY
                    temp = minimax(state, OPPONENT_PLAYS, depthRemaining - 1)
                    default_score = max(default_score, temp)
                    board[row][col] = ' '
    else:
        default_score = float('inf')
        for row in range(N):
            for col in range(M):
                if board[row][col] == ' ':
                    board[row][col] = OPPONENT_PLAYS
                    temp = minimax(state, I_PLAY, depthRemaining - 1)
                    default_score = min(default_score, temp)
                    board[row][col] = ' '
    
    return default_score

minimax(state, I_PLAY, 2)

-11

In [4]:
def makeMove(currentState, currentRemark, timeLimit=10000):

    # Here's a placeholder:
    a_default_move = [0, 0]

    N = len(currentState[0][:][:])
    M = len(currentState[0][0])
    currentboard = copy.copy(currentState[0])
    default_score = 0
    rowBest = 0
    colBest = 0
    n = 2

    for row in range(N):
        for col in range(M):
            
            if currentboard[row][col] == ' ':
                currentboard[row][col] = I_PLAY
                
                temp = minimax(currentState, I_PLAY, n)
                
                if temp > default_score:
                    rowBest = row
                    colBest = col
                    default_score = temp

                currentboard[row][col] = ' '


    a_default_move = [rowBest, colBest]

    newState = copy.deepcopy(currentState)
    newState[0][a_default_move[0]][a_default_move[1]] = I_PLAY
    
    newRemark = "I need to think of something appropriate.\n" +\
    "Well, I guess I can say that this move is probably illegal."

    print(a_default_move)
    return [[a_default_move, newState], newRemark]

makeMove(state, currentRemark=None)[0][1]

[0, 0]


[[['X', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', 'O', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', '-', '-', ' ', ' ', ' '],
  [' ', ' ', '-', '-', '-', '-', ' ', ' '],
  [' ', ' ', ' ', '-', '-', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
  [' ', ' ', ' ', ' ', ' ', ' ', 'O', ' ']],
 'X']